In [1]:
# Shell Plus Model Imports
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

from book.models import Book, Metadata
from django.contrib.admin.models import LogEntry
from django.contrib.auth.models import Group, Permission, User
from django.contrib.contenttypes.models import ContentType
from django.contrib.sessions.models import Session
from taggit.models import Tag, TaggedItem
# Shell Plus Django Imports
from django.core.cache import cache
from django.conf import settings
from django.contrib.auth import get_user_model
from django.db import transaction
from django.db.models import Avg, Case, Count, F, Max, Min, Prefetch, Q, Sum, When
from django.utils import timezone
from django.urls import reverse
from django.db.models import Exists, OuterRef, Subquery

In [2]:
from functools import reduce

In [3]:
tagsPart1 = Book.objects.all()[0].tags.names()
tagsPart2 = Book.objects.all()[1].tags.names()

print(tagsPart1, len(tagsPart1))
print(tagsPart2, len(tagsPart2))
print(tagsPart1.union(tagsPart2, all=True), len(tagsPart1.union(tagsPart2, all=True)))

<QuerySet ['#개발자', '#두잇', '#분철', '#새해결심', '#컴공', '#파이썬', '#프로그래밍', 'IT 모바일', '국내도서', '파이썬', '프로그래밍 언어']> 11
<QuerySet ['#다시보는2020', '2020년 올해의 책 후보도서', 'IT 모바일', 'YES24 올해의 책', '국내도서', '컴퓨터 공학', '컴퓨터 교육']> 7
<QuerySet ['#개발자', '#두잇', '#분철', '#새해결심', '#컴공', '#파이썬', '#프로그래밍', 'IT 모바일', '국내도서', '파이썬', '프로그래밍 언어', '#다시보는2020', '2020년 올해의 책 후보도서', 'IT 모바일', 'YES24 올해의 책', '국내도서', '컴퓨터 공학', '컴퓨터 교육']> 18


In [42]:
totalTags = {}
tidyTagsPart1 = list(set(" ".join(list(tagsPart1)).replace("#", "").split(" ")))
tidyTagsPart1.remove("모바일")
tidyTagsPart1.remove("IT")
tidyTagsPart1.remove("국내도서")
for item in tidyTagsPart1:
    try: 
        totalTags[item] = totalTags[item] + 1
    except:
        totalTags[item] = 1

totalTags

{'새해결심': 1, '프로그래밍': 1, '파이썬': 1, '언어': 1, '컴공': 1, '두잇': 1, '분철': 1, '개발자': 1}

In [51]:
totalTags = {}
for book in Book.objects.all():
    tidyTagsPart = list(set(" ".join(list(book.tags.names())).replace("#", "").split(" ")))
    
    try:
        tidyTagsPart.remove("모바일")
    except:
        pass
    
    try:
        tidyTagsPart.remove("IT")
    except:
        pass
    
    try:
        tidyTagsPart.remove("국내도서")
    except:
        pass
    
    try:
        tidyTagsPart.remove("컴퓨터")
    except:
        pass
    
    try:
        tidyTagsPart.remove("프로그래밍")
    except:
        pass
    
    try:
        tidyTagsPart.remove("컴공")
    except:
        pass
    
    try:
        tidyTagsPart.remove("공학")
    except:
        pass
    
    for item in tidyTagsPart:
        try: 
            totalTags[item] = totalTags[item] + 1
        except:
            totalTags[item] = 1

In [60]:
totalTags
dict(sorted(totalTags.items(), key=lambda x: x[1], reverse=True))

{'언어': 781,
 'OS/데이터베이스': 356,
 '파이썬': 298,
 '그래픽/디자인/멀티미디어': 297,
 '기타': 261,
 '클라우드/빅데이터': 250,
 '인공지능': 238,
 '일반': 234,
 '입문/활용': 228,
 '교육': 223,
 '분철': 212,
 '자료구조/알고리즘': 209,
 '활용': 185,
 '오피스': 185,
 '소프트웨어': 168,
 '웹사이트': 167,
 '어린이': 151,
 '네트워크/해킹/보안': 144,
 '대학교재': 142,
 'MS': 137,
 '인터넷': 133,
 '경영': 133,
 '경제': 133,
 '개발방법론': 118,
 '공학계열': 111,
 '비즈니스': 104,
 '컴퓨터공학': 102,
 '모바일/태블릿/SNS': 100,
 '게임': 100,
 '그래픽일반/자료집': 97,
 'C': 87,
 'Excel(엑셀)': 85,
 '보안/해킹': 84,
 '자바': 79,
 '데이터베이스': 79,
 '시스템관리/서버': 75,
 '마케팅': 70,
 'HTML/JavaScript/CSS/jQuery': 69,
 '네트워크': 64,
 '개발자': 61,
 '개발': 60,
 '마케팅/세일즈': 59,
 'UI/UX': 59,
 '예술': 59,
 'JavaScript/CGI': 58,
 '포토샵': 56,
 'PowerPoint(파워포인트)': 54,
 '인터넷비즈니스': 54,
 '프레젠테이션': 51,
 '안드로이드폰': 51,
 '미술': 50,
 '그래픽툴/저작툴': 49,
 '위한': 47,
 '음악/영상': 47,
 '멀티미디어/컴퓨터': 47,
 '자연과학': 46,
 '어른을': 44,
 'C++': 43,
 '쇼핑몰/창업': 40,
 '자기계발': 40,
 'AUTOCAD': 40,
 '': 39,
 'IT스테디셀러': 39,
 '입문서': 38,
 '통신': 37,
 '네트워크/데이터': 37,
 '웹디자인': 36,
 '리눅스': 36,
 